In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
import mmseg
print(mmseg.__version__)

import mmcv
import random
import os.path as osp
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset
from mmcv import Config
from mmseg.apis import set_random_seed

1.10.0 False
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
0.21.1


In [2]:
#%cd mmsegmentation

In [2]:
data_root = '../dataset/fsd4000'
img_dir = 'Original'
ann_dir = 'Face'
classes = ('face','bg')
palette = [[129, 127, 38],[249, 220, 212]]

@DATASETS.register_module()
class StandfordBackgroundDataset(CustomDataset):
    CLASSES = classes
    PALETTE = palette
    def __init__(self, split, **kwargs):
        super().__init__(img_suffix='.jpg', seg_map_suffix='.png', 
                         split=split, **kwargs)
        assert osp.exists(self.img_dir) and self.split is not None

In [3]:
# Create a config file
cfg = Config.fromfile('mmsegmentation/configs/swin/upernet_swin_tiny_patch4_window7_512x512_160k_ade20k_pretrain_224x224_1K.py')

# Since we use ony one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
#cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 2
cfg.model.auxiliary_head.num_classes = 2


# Modify dataset type and path
cfg.dataset_type = 'StandfordBackgroundDataset'
cfg.data_root = data_root

#batch size
cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 4 

cfg.img_norm_cfg = dict(mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (256, 256)
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(320, 240), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(320, 240),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]


cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = img_dir
cfg.data.train.ann_dir = ann_dir
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = 'face_splits/train.txt'

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = img_dir
cfg.data.val.ann_dir = ann_dir
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = 'face_splits/val.txt'

cfg.data.test.type = cfg.dataset_type
cfg.data.test.data_root = cfg.data_root
cfg.data.test.img_dir = img_dir
cfg.data.test.ann_dir = ann_dir
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = 'face_splits/val.txt'

# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
#cfg.load_from = 'train_skinseg/latest.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './train_faceseg'

cfg.optimizer.lr = 1e-04
#cfg.runner.type = 'EpochBasedRunner'
cfg.runner.max_iters = 400*200
cfg.log_config.interval = 200
##
cfg.evaluation.interval = 400
#
cfg.checkpoint_config.interval = 400*5

cfg.lr_config.warmup_iters = 200
cfg.lr_config.warmup_ratio = 1e-04

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='BN', requires_grad=True)
backbone_norm_cfg = dict(type='LN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained='pretrain/swin_tiny_patch4_window7_224.pth',
    backbone=dict(
        type='SwinTransformer',
        pretrain_img_size=224,
        embed_dims=96,
        patch_size=4,
        window_size=7,
        mlp_ratio=4,
        depths=[2, 2, 6, 2],
        num_heads=[3, 6, 12, 24],
        strides=(4, 2, 2, 2),
        out_indices=(0, 1, 2, 3),
        qkv_bias=True,
        qk_scale=None,
        patch_norm=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.3,
        use_abs_pos_embed=False,
        act_cfg=dict(type='GELU'),
        norm_cfg=dict(type='LN', requires_grad=True)),
    decode_head=dict(
        type='UPerHead',
        in_channels=[96, 192, 384, 768],
        in_index=[0, 1, 2, 3],
        pool_scales=(1, 2, 3, 6),
        channels=512,
        dropout_ratio=0.1,
        num_cla

## Train and Eval

In [5]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, meta=dict({'PALETTE':palette}))

2022-03-07 16:17:14,146 - mmseg - INFO - Loaded 3200 images
/home/ec2-user/.local/lib/python3.7/site-packages/mmseg/models/backbones/swin.py:556: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
2022-03-07 16:17:18,042 - mmseg - INFO - Loaded 800 images
2022-03-07 16:17:18,043 - mmseg - INFO - Start running, host: ec2-user@ip-172-31-35-195.us-east-2.compute.internal, work_dir: /home/ec2-user/blm/code/train_faceseg
2022-03-07 16:17:18,044 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook     

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 800/800, 6.8 task/s, elapsed: 118s, ETA:     0s

2022-03-07 16:27:18,713 - mmseg - INFO - per class results:
2022-03-07 16:27:18,715 - mmseg - INFO - 
+-------+-------+-------+
| Class |  IoU  |  Acc  |
+-------+-------+-------+
|  face | 41.51 | 70.42 |
|   bg  | 89.81 | 92.62 |
+-------+-------+-------+
2022-03-07 16:27:18,716 - mmseg - INFO - Summary:
2022-03-07 16:27:18,717 - mmseg - INFO - 
+------+-------+-------+
| aAcc |  mIoU |  mAcc |
+------+-------+-------+
| 90.5 | 65.66 | 81.52 |
+------+-------+-------+
2022-03-07 16:27:18,723 - mmseg - INFO - Iter(val) [800]	aAcc: 0.9050, mIoU: 0.6566, mAcc: 0.8152, IoU.face: 0.4151, IoU.bg: 0.8981, Acc.face: 0.7042, Acc.bg: 0.9262
2022-03-07 16:31:21,960 - mmseg - INFO - Iter [600/80000]	lr: 9.925e-05, eta: 1 day, 7:00:52, time: 1.807, data_time: 0.607, memory: 4940, decode.loss_ce: 0.2405, decode.acc_seg: 74.7382, aux.loss_ce: 0.1037, aux.acc_seg: 73.8562, loss: 0.3442
2022-03-07 16:35:23,245 - mmseg - INFO - Iter [800/80000]	lr: 9.900e-05, eta: 1 day, 5:50:15, time: 1.206, data_tim

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 800/800, 6.8 task/s, elapsed: 118s, ETA:     0s

2022-03-07 16:37:21,135 - mmseg - INFO - per class results:
2022-03-07 16:37:21,136 - mmseg - INFO - 
+-------+-------+-------+
| Class |  IoU  |  Acc  |
+-------+-------+-------+
|  face | 44.36 | 69.42 |
|   bg  | 91.07 | 94.02 |
+-------+-------+-------+
2022-03-07 16:37:21,137 - mmseg - INFO - Summary:
2022-03-07 16:37:21,138 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 91.66 | 67.71 | 81.72 |
+-------+-------+-------+
2022-03-07 16:37:21,147 - mmseg - INFO - Iter(val) [800]	aAcc: 0.9166, mIoU: 0.6771, mAcc: 0.8172, IoU.face: 0.4436, IoU.bg: 0.9107, Acc.face: 0.6942, Acc.bg: 0.9402
2022-03-07 16:41:24,364 - mmseg - INFO - Iter [1000/80000]	lr: 9.875e-05, eta: 1 day, 7:44:03, time: 1.806, data_time: 0.605, memory: 4940, decode.loss_ce: 0.2222, decode.acc_seg: 76.8722, aux.loss_ce: 0.0961, aux.acc_seg: 76.0210, loss: 0.3183
2022-03-07 16:45:25,431 - mmseg - INFO - Iter [1200/80000]	lr: 9.850e-05, eta: 1 day, 6:46:31, time: 1.205, d

...

2022-03-09 01:41:18,179 - mmseg - INFO - per class results:
2022-03-09 01:41:18,180 - mmseg - INFO - 
+-------+-------+-------+
| Class |  IoU  |  Acc  |
+-------+-------+-------+
|  face | 79.68 | 88.02 |
|   bg  | 97.65 | 98.89 |
+-------+-------+-------+
2022-03-09 01:41:18,181 - mmseg - INFO - Summary:
2022-03-09 01:41:18,182 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 97.85 | 88.67 | 93.46 |
+-------+-------+-------+
2022-03-09 01:41:18,191 - mmseg - INFO - Iter(val) [800]	aAcc: 0.9785, mIoU: 0.8867, mAcc: 0.9346, IoU.face: 0.7968, IoU.bg: 0.9765, Acc.face: 0.8802, Acc.bg: 0.9889
2022-03-09 01:45:23,104 - mmseg - INFO - Iter [79400/80000]	lr: 7.513e-07, eta: 0:15:10, time: 1.822, data_time: 0.613, memory: 4940, decode.loss_ce: 0.0540, decode.acc_seg: 83.5442, aux.loss_ce: 0.0265, aux.acc_seg: 83.1461, loss: 0.0805
2022-03-09 01:49:26,059 - mmseg - INFO - Iter [79600/80000]	lr: 5.012e-07, eta: 0:10:06, time: 1.215, data_time: 0.

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 800/800, 6.7 task/s, elapsed: 120s, ETA:     0s

2022-03-09 01:51:25,940 - mmseg - INFO - per class results:
2022-03-09 01:51:25,942 - mmseg - INFO - 
+-------+-------+-------+
| Class |  IoU  |  Acc  |
+-------+-------+-------+
|  face |  79.9 | 87.83 |
|   bg  | 97.69 | 98.95 |
+-------+-------+-------+
2022-03-09 01:51:25,943 - mmseg - INFO - Summary:
2022-03-09 01:51:25,944 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 97.88 | 88.79 | 93.39 |
+-------+-------+-------+
2022-03-09 01:51:25,952 - mmseg - INFO - Iter(val) [800]	aAcc: 0.9788, mIoU: 0.8879, mAcc: 0.9339, IoU.face: 0.7990, IoU.bg: 0.9769, Acc.face: 0.8783, Acc.bg: 0.9895
2022-03-09 01:55:30,518 - mmseg - INFO - Iter [79800/80000]	lr: 2.513e-07, eta: 0:05:03, time: 1.822, data_time: 0.615, memory: 4940, decode.loss_ce: 0.0523, decode.acc_seg: 82.6816, aux.loss_ce: 0.0263, aux.acc_seg: 82.2213, loss: 0.0786
2022-03-09 01:59:59,397 - mmseg - INFO - Saving checkpoint at 80000 iterations
2022-03-09 02:00:01,451 - mmseg - IN

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 800/800, 6.5 task/s, elapsed: 124s, ETA:     0s

2022-03-09 02:02:05,344 - mmseg - INFO - per class results:
2022-03-09 02:02:05,345 - mmseg - INFO - 
+-------+-------+-------+
| Class |  IoU  |  Acc  |
+-------+-------+-------+
|  face |  79.8 | 87.96 |
|   bg  | 97.67 | 98.92 |
+-------+-------+-------+
2022-03-09 02:02:05,346 - mmseg - INFO - Summary:
2022-03-09 02:02:05,347 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 97.87 | 88.74 | 93.44 |
+-------+-------+-------+
2022-03-09 02:02:05,355 - mmseg - INFO - Iter(val) [800]	aAcc: 0.9787, mIoU: 0.8874, mAcc: 0.9344, IoU.face: 0.7980, IoU.bg: 0.9767, Acc.face: 0.8796, Acc.bg: 0.9892
